In [ ]:
import os
import re
import sys
import glob
import pickle
import numpy as np
from scipy.fft import fft, fftfreq
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.transforms as mtransforms
from matplotlib.gridspec import GridSpec

fontsize = 8
lw = 0.75

matplotlib.rc('font', **{'family': 'Times New Roman', 'size': fontsize})
matplotlib.rc('axes', **{'linewidth': 0.75, 'labelsize': fontsize})
matplotlib.rc('xtick', **{'labelsize': fontsize})
matplotlib.rc('ytick', **{'labelsize': fontsize})
matplotlib.rc('xtick.major', **{'width': lw, 'size':3})
matplotlib.rc('ytick.major', **{'width': lw, 'size':3})
matplotlib.rc('ytick.minor', **{'width': lw, 'size':1.5})

%matplotlib inline

if '..' not in sys.path:
    sys.path.append('..')
from dlml.data import load_data_files, load_data_areas

In [ ]:
def make_axes(rows, cols, x_offset, y_offset, x_space, y_space, squeeze=True):
    w = (1 - np.sum(x_offset) - x_space * (cols - 1)) / cols
    h = (1 - np.sum(y_offset) - y_space * (rows - 1)) / rows
    
    ax = [[plt.axes([x_offset[0] + (w + x_space) * j,
                     y_offset[0] + (h + y_space) * i,
                     w, h]) for j in range(cols)] for i in range(rows-1, -1, -1)]
    
    for row in ax:
        for a in row:
            for side in 'right','top':
                a.spines[side].set_visible(False)

    if squeeze:
        if rows == 1 and cols == 1:
            return ax[0][0]
        if rows == 1:
            return ax[0]
        if cols == 1:
            return [a[0] for a in ax]
        
    return ax

In [ ]:
X, y, Xf = {}, {}, {}
group_index, n_mom_groups = {}, {}

In [ ]:
# data_file = 'traces_hist_spectra.npz'
data_file = 'traces_hist_spectra_comp_grid.npz'
force = False
if not os.path.isfile(data_file) or force:
    if data_file == 'traces_hist_spectra.npz':
        set_name = 'test'
    else:
        set_name = 'training'
    data_dir = '../data/IEEE39/converted_from_PowerFactory/all_stoch_loads/var_H_area_1'
    data_files = sorted(glob.glob(data_dir + os.path.sep + f'*_{set_name}_set.h5'))
    var_names = ['Vd_bus3']
    generators_areas_map = [['G02', 'G03'], ['G04', 'G05', 'G06', 'G07'], ['G08', 'G09', 'G10'], ['G01']]
    generators_Pnom = {'G01': 10e9, 'G02': 700e6, 'G03': 800e6, 'G04': 800e6, 'G05': 300e6,
                       'G06': 800e6, 'G07': 700e6, 'G08': 700e6, 'G09': 1000e6, 'G10': 1000e6}
    area_measure = 'momentum'
    ret = load_data_areas({set_name: data_files}, var_names,
                          generators_areas_map[:1],
                          generators_Pnom,
                          area_measure,
                          trial_dur=60,
                          max_block_size=1000,
                          use_tf=False,
                          add_omega_ref=True,
                          use_fft=False)
    t = ret[0]
    X_raw = ret[1][set_name]
    y[set_name] = ret[2][set_name]
    group_index[set_name] = [np.where(y[set_name] == mom)[0] for mom in np.unique(y[set_name])]
    n_mom_groups[set_name] = len(group_index[set_name])
    X_mean, X_std = X_raw.mean(axis=(1,2)), X_raw.std(axis=(1,2))
    X[set_name] = (X_raw - X_mean) / X_std
    X[set_name] = X[set_name].squeeze()
    y[set_name] = y[set_name].squeeze()

    dt = np.diff(t[:2])[0]
    N_samples = t.size
    Xf[set_name] = fft(X[set_name])
    Xf[set_name] = 2.0 / N_samples * np.abs(Xf[set_name][:, :N_samples//2])
    F = fftfreq(N_samples, dt)[:N_samples//2]


    data_dir = '../data/IEEE39/converted_from_PowerFactory/all_stoch_loads/low_momentum_test_var_G1_G2'
    data_files = sorted(glob.glob(data_dir + os.path.sep + '*.h5'))
    idx = list(map(lambda f: len(re.findall('_[0-9]*.h5', f)) > 0, data_files))
    data_files = [data_files[j] for j,i in enumerate(idx) if i]
    N = len(np.unique([int(os.path.splitext(re.findall('_[0-9]*.h5', f)[0])[0][1:]) for f in data_files]))
    M = len(data_files) // N
    group_index['var_G2_G3'] = np.reshape(np.arange(len(data_files)), (M,N))

    ret = load_data_files(data_files,
                          var_names,
                          generators_areas_map[:1],
                          generators_Pnom,
                          'momentum')

    X_raw = ret[1][:, :, :-1]
    X['var_G2_G3'] = np.zeros(X_raw.shape)
    X['var_G2_G3'] = (X_raw - X_mean) / X_std
    y['var_G2_G3'] = ret[2]
    X['var_G2_G3'] = X['var_G2_G3'].squeeze()
    y['var_G2_G3'] = y['var_G2_G3'].squeeze()
    Xf['var_G2_G3'] = fft(X['var_G2_G3'])
    Xf['var_G2_G3'] = 2.0 / N_samples * np.abs(Xf['var_G2_G3'][:, :N_samples//2])
    n_mom_groups['var_G2_G3'] = len(group_index['var_G2_G3'])
    
    np.savez_compressed(data_file, t=t, X=X, y=y, Xf=Xf, F=F,
                        group_index=group_index, n_mom_groups=n_mom_groups)
else:
    if data_file == 'traces_hist_spectra.npz':
        set_name = 'test'
    else:
        set_name = 'training'
    data = np.load(data_file, allow_pickle=True)
    t = data['t']
    X = data['X'].item()
    y = data['y'].item()
    F = data['F']
    Xf = data['Xf'].item()
    group_index = data['group_index'].item()
    n_mom_groups = data['n_mom_groups'].item()

In [ ]:
fig,ax = plt.subplot_mosaic(
    '''
    AAAAAAAA
    BBBBBCCC
    DDDDEEEE
    ''',
    figsize=(3.5,4)
)
cmap = plt.get_cmap('Set2')
N_div_cmap = 10
div_cmap = plt.get_cmap('bwr', N_div_cmap)

###########################################
momentum = lambda H, S, fn: 2 * H@S / fn * 1e-3

N = 11, 11
h_G1_0, h_G2_0 = 4.33, 4.47
h_G1 = h_G1_0 + np.linspace(-1, 1, N[0])
h_G2 = h_G2_0 + np.linspace(-1, 1, N[1])
H_G1, H_G2 = np.meshgrid(*[h_G1, h_G2])

M = np.zeros(N)
S = np.array([700, 800])
fn = 60
for i in range(N[0]):
    for j in range(N[1]):
        H = np.array([H_G1[i,j], H_G2[i,j]])
        M[i,j] = momentum(H, S, fn)
        
gray_cmap = plt.get_cmap('gray')
cont = ax['A'].contourf(H_G1, H_G2, M, levels=100, cmap=gray_cmap)
cbar = plt.colorbar(cont, ax=ax['A'])
white = [1,1,1]
magenta = [1,0,1]
green = [0,1,0]
yellow = [1,1,0]
blue = [0,.5,1]
red = [1,.333,.333]
orange = [1, .5, 0]
gray = [.6, .6, .6]
ax['A'].scatter(H_G1[::2, ::2], H_G2[::2, ::2], s=5, c='w', edgecolors='k', lw=0.5, marker='o')
for i in range(2):
    for j in range(2):
        ax['A'].plot(H_G1[i,j], H_G2[i,j], 's', markersize=4, lw=1,
                    color=div_cmap(i*2+j), markerfacecolor='none')
        ax['A'].plot(H_G1[-1-i,-1-j], H_G2[-1-i,-1-j], 's', markersize=4, lw=1,
                    color=div_cmap(N_div_cmap-1-i*2-j), markerfacecolor='none')
for i in range(0, N[0], 2):
    ax['A'].plot(H_G1[i,i], H_G2[i,i], 'o', color=cmap(i//2), markerfacecolor='none', markersize=3, lw=1)
ax['A'].plot(H_G1[:2,:2].mean(), H_G2[:2,:2].mean(), 'x', color=magenta, markersize=4, markeredgewidth=1)
ax['A'].plot(H_G1[-2:,-2:].mean(), H_G2[-2:,-2:].mean(), 'x', color=magenta, markersize=4, markeredgewidth=1)
ax['A'].set_xlabel(r'$H_{G_1}$ [s]')
ax['A'].set_ylabel(r'$H_{G_2}$ [s]')
ax['A'].set_xlim([h_G1_0 - 1.1, h_G1_0 + 1.1])
ax['A'].set_ylim([h_G2_0 - 1.1, h_G2_0 + 1.1])
ax['A'].set_xticks([h_G1_0 - 1, h_G1_0, h_G1_0 + 1])
ax['A'].set_yticks([h_G2_0 - 1, h_G2_0, h_G2_0 + 1])
cbar.set_label(r'Momentum [GW$\cdot$s]')
cbar.set_ticks(np.r_[0.17 : 0.28 : 0.02])
###########################################

tend = 10
jdx, = np.where(t < tend)
for i,idx in enumerate(group_index[set_name]):
    n,edges = np.histogram(X[set_name][idx,:], bins=50, density=True)
    ax['B'].plot(t[jdx], X[set_name][idx[0]+1, jdx], lw=1, color=cmap(i))
    ax['C'].plot(n, edges[1:], lw=1, color=cmap(i))
    ax['D'].plot(F, 20 * np.log10(Xf[set_name][idx,:].mean(axis=0)), lw=1,
               color=cmap(i), label=r'{:.3f} GW$\cdot$s'.format(y[set_name][idx[0]+1]))

for i,idx in enumerate(group_index['var_G2_G3']):
    if i < 4:
        ax['E'].plot(F, 20 * np.log10(Xf['var_G2_G3'][idx,:].mean(axis=0)), lw=1,
                   color=div_cmap(i))
    else:
        ax['E'].plot(F, 20 * np.log10(Xf['var_G2_G3'][idx,:].mean(axis=0)), lw=1,
                   color=div_cmap(N_div_cmap - (i - 3)))

for key in 'BCDE':
    ax[key].grid(which='major', axis='y', lw=0.5, ls=':', color=[.6,.6,.6])
for a in ax.values():
    for side in 'right','top':
        a.spines[side].set_visible(False)
for key in 'BC':
    ax[key].set_ylim([-3.5, 3.5])
    ax[key].set_yticks(np.r_[-3 : 4 : 1.5])
for key in 'DE':
    ax[key].set_ylim([-55, -5])
    ax[key].set_yticks(np.r_[-50 : -5 : 20])
    ax[key].set_xscale('log')
    ax[key].set_xlabel('Frequency [Hz]')
ax['C'].set_yticklabels([])
ax['E'].set_yticklabels([])

ax['B'].set_xlabel('Time [s]')
ax['B'].set_ylabel('Norm. V')
ax['C'].set_xlabel('Distr.')
ax['D'].set_ylabel('Power [dB]')

trans = mtransforms.ScaledTranslation(-0.45, -0.05, fig.dpi_scale_trans)
for label in 'ABD':
    ax[label].text(0.0, 1.0, label, transform=ax[label].transAxes + trans, fontsize=10, va='bottom')
trans = mtransforms.ScaledTranslation(-0.15, -0.05, fig.dpi_scale_trans)
for label in 'CE':
    ax[label].text(0.0, 1.0, label, transform=ax[label].transAxes + trans, fontsize=10, va='bottom')

fig.tight_layout(pad=0)
plt.savefig('traces_hist_spectra_comp_grid.pdf')